In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [7]:
from geo_tool import Point_Cloud
from tf_lab.evaluate.generative_pc_nets import minimum_mathing_distance
from numpy.random import normal
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.in_out import PointCloudDataSet, load_point_clouds_from_filenames
from tf_lab.data_sets.shape_net import pc_loader as snc_loader

def load_multiple_version_of_pcs(version, syn_id, n_classes, n_pc_points=2048, random_seed=42, verbose=False):
    top_data_dir = '/orions4-zfs/projects/optas/DATA/'

    if n_classes != 1:
        raise ValueError()

    elif version == 'uniform_one':
#         versions = ['centered_2nd_version']
        versions = ['centered_3rd_version']


    splits = {'train': None, 'val': None, 'test': None}

    for s in splits.keys():
        if verbose:
            print 'Loading %s data.' % (s,)
        s_file = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/' + syn_id + '/85_5_10/', s + '.txt')
        if verbose:
            print s_file
        for _, v in enumerate(versions):
            top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', v, str(n_pc_points))
            splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
            pcs_in_split = splitter.load_splits(s_file)
            pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(pcs_in_split, n_threads=20, loader=snc_loader, verbose=True)
            if splits[s] is None:
                splits[s] = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
            else:
                splits[s].merge(PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids))
    return splits

In [10]:
class_name = 'chair'
syn_id = snc_category_to_synth_id()[class_name]
pc1 = load_multiple_version_of_pcs('uniform_one', syn_id, 1)

In [13]:
# manually change versions
pc2 = load_multiple_version_of_pcs('uniform_one', syn_id, 1)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [15]:
a = pc1['val'].point_clouds
# b = pc1['val'].point_clouds + normal(scale=0.01, size=(pc1['val'].point_clouds.shape))
b = pc2['val'].point_clouds
minimum_mathing_distance(a, b, 400, use_EMD=True)


(67.613182,
 [68.932503,
  59.148788,
  57.316463,
  81.253998,
  72.036713,
  54.369331,
  52.135326,
  54.576813,
  91.749901,
  53.416336,
  74.026047,
  90.837967,
  53.733208,
  75.031334,
  87.387222,
  74.902649,
  57.649536,
  74.698868,
  60.144669,
  54.087166,
  73.962784,
  84.798904,
  58.506214,
  79.375458,
  61.237816,
  64.998566,
  68.484985,
  68.553085,
  68.142609,
  57.576447,
  86.003342,
  88.05983,
  75.841812,
  91.844635,
  65.591904,
  63.844154,
  62.638134,
  59.298576,
  65.096878,
  58.709827,
  83.181946,
  87.819656,
  68.303322,
  51.957462,
  51.151226,
  67.248009,
  72.380165,
  71.576164,
  64.338806,
  70.089249,
  66.021179,
  63.459946,
  58.537441,
  56.556511,
  79.697929,
  66.435242,
  99.43399,
  62.664803,
  60.131531,
  65.346939,
  67.112717,
  58.536266,
  80.535355,
  60.299301,
  58.245609,
  88.135849,
  67.020828,
  54.943905,
  58.882904,
  50.480225,
  72.146263,
  93.604263,
  71.268692,
  75.445923,
  51.519318,
  64.577003,
  

In [9]:
sample_files = ['/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/decoded_gt_data_aligned_to_gt/chair/mlp_with_split_1pc_usampled_bnorm_on_encoder_only/emd_bneck_128/train.npz']

In [10]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [ ]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file, boost_samples)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
        f_mmd.flush()
    print

train
train


In [12]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()